In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ast

ERROR: Could not find a version that satisfies the requirement ast (from versions: 0.0.1, 0.0.2)
ERROR: No matching distribution found for ast


In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/movies_metadata_fixed.csv')
df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415


In [5]:
df.shape

(45463, 24)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45463 non-null  float64
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re



---

# Drop columns

---



In [7]:
df = df[['title', 'release_date', 'vote_count', 'vote_average', 'genres']]
df.head()

,title,release_date,vote_count,vote_average,genres
0,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,1995-12-15,2413,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,1995-12-22,92,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,1995-12-22,34,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,1995-02-10,173,5.7,"[{'id': 35, 'name': 'Comedy'}]"




---

# Handling the 'genres' column

---



In [8]:
# Transform JSON-format to python array
def handling_genres_column_json(df):
  df['genres'] = df['genres'].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  return df

In [9]:
# Analytics of 'genres' columns
def value_counts_for_genres(df):
  genres = []
  count_of_oblects = df.shape[0]
  for i in range(count_of_oblects):
    for k in df.iloc[i][4]:
      if k not in genres:
        genres.append(k)
  return genres

In [ ]:
'''def drop_list_genres(df):
  df.genres = [', '.join(i) for i in df.genres]
  return df'''

In [11]:
df = handling_genres_column_json(df)
df.head(1)

,title,release_date,vote_count,vote_average,genres
0,Toy Story,1995-10-30,5415,7.7,"[Animation, Comedy, Family]"


In [12]:
value_counts_for_genres(df)

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'History',
 'Science Fiction',
 'Mystery',
 'War',
 'Foreign',
 'Music',
 'Documentary',
 'Western',
 'TV Movie']

In [ ]:
'''df = drop_list_genres(df)'''

In [14]:
df.head()

,title,release_date,vote_count,vote_average,genres
0,Toy Story,1995-10-30,5415,7.7,"[Animation, Comedy, Family]"
1,Jumanji,1995-12-15,2413,6.9,"[Adventure, Fantasy, Family]"
2,Grumpier Old Men,1995-12-22,92,6.5,"[Romance, Comedy]"
3,Waiting to Exhale,1995-12-22,34,6.1,"[Comedy, Drama, Romance]"
4,Father of the Bride Part II,1995-02-10,173,5.7,[Comedy]


In [15]:
mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('genres')),
                index=df.index,
                columns=mlb.classes_))

In [16]:
df.head()

,title,release_date,vote_count,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Toy Story,1995-10-30,5415,7.7,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Jumanji,1995-12-15,2413,6.9,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,1995-12-22,92,6.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Waiting to Exhale,1995-12-22,34,6.1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Father of the Bride Part II,1995-02-10,173,5.7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0




---

# General Rating Recommendation System

---



In [17]:
# To avoid infinity in the "weights" column
df = df[df.vote_count != 0]

In [18]:
df['weight'] = df['vote_count'] / df['vote_average']
df = df[df.vote_average != 0]

In [19]:
df.sort_values('vote_average', ascending=False).head()

,title,release_date,vote_count,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,weight
37332,Book of Days,1989-11-30,1,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1
15249,Ween: Live in Chicago,2004-05-04,1,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1
15397,Little Dorrit,1987-12-11,1,10.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1
33278,Dream Deceivers: The Story Behind James Vance ...,1992-08-06,1,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1
45359,The Many Faces of Christopher Lee,1996-01-01,1,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1


In [20]:
# Output is 50 best movies in different genres
def general_vote(df):
  return df.sort_values('weight', ascending=False).head(50)

In [21]:
general_vote(df)

,title,release_date,vote_count,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,weight
15480,Inception,2010-07-14,14075,8.1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1737.654321
14551,Avatar,2009-12-10,12114,7.2,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1682.500000
17818,The Avengers,2012-04-25,12000,7.4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1621.621622
26563,Deadpool,2016-02-09,11444,7.4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546.486486
12481,The Dark Knight,2008-07-16,12269,8.3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1478.192771
18244,The Hunger Games,2012-03-12,9634,6.9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1396.231884
22878,Interstellar,2014-11-05,11187,8.1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1381.111111
25083,Jurassic World,2015-06-09,8842,6.5,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1360.307692
20050,Django Unchained,2012-12-25,10297,7.8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1320.128205
26552,Mad Max: Fury Road,2015-05-13,9629,7.3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1319.041096




---

# Rating Recommender System by Genre

---



In [23]:
# Output is 50 best movies in definite genre
def genres_vote(df, genre):
  df1 = df[df[genre] == 1]
  return df1.sort_values('weight', ascending=False).head(50)

In [24]:
genres_vote(df, 'Crime')

,title,release_date,vote_count,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,weight
12481,The Dark Knight,2008-07-16,12269,8.3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1478.192771
30541,Suicide Squad,2016-08-02,7717,5.9,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1307.966102
18252,The Dark Knight Rises,2012-07-16,9263,7.6,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1218.815789
292,Pulp Fiction,1994-09-10,8670,8.3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1044.578313
10122,Batman Begins,2005-06-10,7511,7.5,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1001.466667
314,The Shawshank Redemption,1994-09-23,8358,8.5,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,983.294118
22130,The Wolf of Wall Street,2013-12-25,6768,7.9,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,856.708861
14557,Sherlock Holmes,2009-12-23,5883,7.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,840.428571
25533,Kingsman: The Secret Service,2015-01-29,6069,7.6,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,798.552632
20969,Fast & Furious 6,2013-05-06,5282,6.7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,788.358209


In [25]:
genres_vote(df, 'Family')

,title,release_date,vote_count,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,weight
4766,Harry Potter and the Philosopher's Stone,2001-11-16,7188,7.5,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,958.400000
15472,Despicable Me,2010-07-08,6595,7.1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,928.873239
13724,Up,2009-05-13,7048,7.8,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,903.589744
30313,Inside Out,2015-06-09,6737,7.9,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,852.784810
6232,Finding Nemo,2003-05-30,6292,7.6,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,827.894737
12704,WALL·E,2008-06-22,6439,7.8,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,825.512821
4756,"Monsters, Inc.",2001-11-01,6150,7.5,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,820.000000
42219,Beauty and the Beast,2017-03-16,5530,6.8,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,813.235294
24454,Big Hero 6,2014-10-24,6289,7.8,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,806.282051
5678,Harry Potter and the Chamber of Secrets,2002-11-13,5966,7.4,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,806.216216
